### Translate
    # addsub
    # multiarith
    # singleeq

In [ ]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import argparse
from tqdm import tqdm
import re

In [ ]:
# PROMPT = {
#     '3p' : get_3p_prompt,
#     'cot' : get_CoT_prompt,
# }

In [ ]:
api = 'openai'
api_key_file = '/userhomes/philhoon/kt-ai-challenge/api-openai.yaml'

In [ ]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0

In [ ]:
input_file = '/userhomes/philhoon/kt-ai-challenge/eng_data/D_multiarith.json'

In [ ]:
output_file_name = 'kor-' + input_file.split('/')[-1]

In [ ]:
output_path = '/userhomes/philhoon/kt-ai-challenge/eng_data'
output_file = output_path + '/' + output_file_name
print(output_file)

In [ ]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [ ]:
# output_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
# print(f'output_file : {output_file}')

In [ ]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [ ]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [ ]:
# Jsonlines writier
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
data = utils.read_json_file(input_file)

In [ ]:
from pprint import pprint

In [ ]:
data = data['data']

In [ ]:
def remove_line_split(context):
    context = re.sub(r'\n{1,}', ' ', context)
    context = re.sub(r'\s{2,}', ' ', context)
    return context

In [ ]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [ ]:
def get_trans_msg(context):
    """
    Message for chat api
    """

    msg = [
        {
            "role" : "system",
            "content" : "Translate given setences into Korean."
        },
        {
            "role" : "user",
            "content" : context
        }
    ]
    
    return msg

In [ ]:
from tqdm import tqdm

In [ ]:
cnt = 0
for ins in tqdm(data.values()):
    instance = ins[0]

#     pprint(cur_ins)
    
    eng_comp = remove_line_split(instance['reasoning_completion'])
    eng_query = instance['question']
    
    query_msg = get_trans_msg(eng_query)
    ans_msg = get_trans_msg(eng_comp)
    
    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
    ans_completion = generate_answer(ans_msg, model_name, max_token, temperature)
    ans_response = ans_completion["choices"][0]["message"]["content"]
    
    instance['kor_query'] = q_response
    instance['kor_ans'] = ans_response
    pprint(instance)
#     jsonl_writer.write_json_line(ins)a
